In [2]:
import re
# from threading import Thread
from multiprocessing import Process
from multiprocessing import Manager
import requests
import time
import xlrd
from bs4 import BeautifulSoup
from lxml import etree
from urllib.parse import quote

In [3]:
root = 'http://www.webofknowledge.com/'
s = requests.get(root)
sid = re.findall(r'SID=\w+&', s.url)[0].replace('SID=', '').replace('&', '')

In [4]:
hearders = {
            'Origin': 'https://apps.webofknowledge.com',
            'Referer': 'https://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch',
            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
            'Content-Type': 'application/x-www-form-urlencoded'
        }

In [5]:
def from_data(sid, SO_name):
    fd = {
            'fieldCount': 3,
            'action': 'search',
            'product': 'WOS',
            'search_mode': 'GeneralSearch',
            'SID': sid,
            'max_field_count': 25,
            'formUpdated': 'true',
            'value(input1)': SO_name,
            'value(select1)': 'SO',
            'value(bool_1_2)': 'AND',
            'value(input2)': 'Article### Review',
            'value(select2)': 'DT',
            'value(bool_2_3)': 'AND',
            'value(input3)': '2017-2018',
            'value(select3)': 'PY',  
            'value(hidInput1)': '',
            'value(hidInput2)': '',
            'limitStatus': 'collapsed',
            'ss_lemmatization': 'On',
            'ss_spellchecking': 'Suggest',
            'SinceLastVisit_UTC': '',
            'SinceLastVisit_DATE': '',
            'period': 'Range Selection',
            'range': 'ALL',
            'startYear': '1975',
            'endYear': '2019',
            'update_back2search_link_param': 'yes',
            'ssStatus': 'display:none',
            'ss_showsuggestions': 'ON',
            'ss_query_language': 'auto',
            'ss_numDefaultGeneralSearchFields': 1,
            'rs_sort_by': 'PY.D;LD.D;SO.A;VL.D;PG.A;AU.A'
        }
    return fd

In [6]:
root_url = 'https://apps.webofknowledge.com/WOS_GeneralSearch.do'

In [7]:
SO_name="FRONTIERS OF STRUCTURAL AND CIVIL ENGINEERING"

In [8]:
s = requests.Session()
r = s.post(root_url, data=from_data(sid, SO_name), headers=hearders)
r.encoding = r.apparent_encoding
tree = etree.HTML(r.text)
soup = BeautifulSoup(r.text, 'lxml')

In [79]:
k=''
for i in tree.xpath("//div[@class='search-results-content']/div/div/a//text()"):
    k+=i
print(k)


Non linear modeling of three-dimensional reinforced and fiber concrete structures

Effect of embedment length of untreated natural fibres on the bond behaviour in cement mortar

Three-scale stochastic homogenization of elastic recycled aggregate concrete based on nano-indentation digital images

Jack up reliability analysis: An overview

Temperature segregation and its impact on the quality and performance of asphalt pavements

Nonlinear analysis and reliability of metallic truss structures

A new fracture criterion for peridynamic and dual-horizon peridynamics

Experimental and numerical analysis of beam to column joints in steel structures

The defect-length effect in corrosion detection with magnetic method for bridge cables

The effect of micro-structural uncertainties of recycled aggregate concrete on its global stochastic properties via finite pixel-element Monte Carlo simulation



In [9]:
paper_num = int(soup.find('span', attrs={'id': 'footer_formatted_count'}).get_text().replace(',', ''))
print(paper_num)

100


In [10]:
url=soup.find('a', class_='snowplow-citation-report citation-report-summary-link')['href']
wos_url='https://apps.webofknowledge.com'
CR_url=wos_url+url
print(CR_url)
cr_r = requests.get(CR_url, 3)
cr_r.encoding = cr_r.apparent_encoding
tree2 = etree.HTML(cr_r.text)
soup2 = BeautifulSoup(cr_r.text, 'lxml')

https://apps.webofknowledge.com/CitationReport.do?product=WOS&search_mode=CitationReport&SID=5CEtCHrcyrUirKesVea&page=1&cr_pqid=1&viewType=summary&colName=WOS


In [11]:
table=soup2.find_all('td', class_='quarter-width')
Total_Publications=table[0].find('em', attrs={'class': 'commafy last'}).get_text().replace(',', '')
h_index = table[1].find('em', attrs={'id': 'H_INDEX'}).get_text().replace(',', '')
Average_citations_per_item=table[1].find('em', attrs={'class': 'minor commafy last'}).get_text().replace(',', '')
Sum_of_Times_Cited=table[2].find('em', attrs={'class': 'commafy'}).get_text().replace(',', '')
without_self_citations=table[2].find('em', attrs={'class': 'minor commafy last'}).get_text().replace(',', '')
Citing_articles=table[3].find('em', attrs={'class': 'commafy'}).get_text().replace(',', '')
without_self_citations2=table[3].find('em', attrs={'class': 'minor commafy last'}).get_text().replace(',', '')

print(Total_Publications, h_index, Average_citations_per_item, Sum_of_Times_Cited, without_self_citations, Citing_articles, without_self_citations2)

100 6 1.85 185 181 171 169


In [12]:
CitReportHead=soup2.find('div', class_='CitReportTotalRow1')
CitReportTable=soup2.find('div', class_='CitReportTotalRow2')
CitYear=[]
CitYearValue=[]
for i in CitReportHead.find_all('div', class_='tcPerYear'):
    CitYear.append(i.get_text().strip())
for i in CitReportTable.find_all('div', class_='totalRow'):
    CitYearValue.append(i.get_text().strip())
print(CitYear, '\n',CitYearValue)

['2016', '2017', '2018', '2019', '2020'] 
 ['0', '12', '55', '118', '0', '185', '61.67']


In [13]:
Citing_articles_href=soup2.find('a', class_='margin-left-20 snowplow-cited-rep-analyze-results-citing-articles')['href']
wos_url='https://apps.webofknowledge.com'
Citing_articles_href=wos_url+Citing_articles_href
qid = re.findall(r'qid=\d+&', Citing_articles_href)[0].replace('qid=', '').replace('&', '')
qid=int(qid)
print(qid)
print(Citing_articles_href)
r3 = requests.get(Citing_articles_href, 3)
r3.encoding = r3.apparent_encoding
tree3 = etree.HTML(r3.text)
soup3 = BeautifulSoup(r3.text, 'lxml')

2
https://apps.webofknowledge.com/TotalCitingArticles.do?product=WOS&search_mode=TotalCitingArticles&qid=2&action=totalCA&SID=5CEtCHrcyrUirKesVea&viewType=summary&betterCount=171&nextRelayAction=GoToRA


In [14]:
headers2 = {
    'host':'apps.webofknowledge.com',
    'Origin': 'https://wcs.webofknowledge.com',
    'Referer': 'https://wcs.webofknowledge.com/RA/analyze.do?product=WOS&SID='+sid+'&field=PY_PublicationYear_PublicationYear_en&yearSort=true',
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    'Content-Type': 'application/x-www-form-urlencoded'
        }
fd2 = {
    'SID': sid,
    'HISTORY':'<b>DOCUMENT TYPES:</b> (ARTICLE OR REVIEW) AND <b>PUBLICATION YEARS:</b> (2018 OR 2017)',
    'queryString':'Total Citing Articles:  PUBLICATION NAME: (frontiers of structural and civil engineering)',
    'action': 'search',
    'product': 'WOS',
    'PRODUCT': 'WOS',
    'canRun':'yes',
    'mode': 'Analyze',
    'viewType':'summary',
    'parentQid': qid+1,
    'resultSetID':qid+1,
    'ua_url':'https://apps.webofknowledge.com',
    'set_base_url':'https://wcs.webofknowledge.com:443/RA',
    'maxSave':100000,
    'queryOption(summary_search_mode)':'TotalCitingArticles',
    'count':171,
    'recordString':171,
    'field':'PY_PublicationYear_PublicationYear_en',
    'recordsStr':'records',
    'ViewRecordsStr':'View Records',
    'visualization':'Bargraph',
    'showNoofResults':25,
    'maxRows':25,
    'refineSelection':'PublicationYear_2019',
        }

In [15]:
s = requests.Session()
url='https://apps.webofknowledge.com/InboundService.do'
r4 = s.post(url, data=fd2, headers=headers2)
r4.encoding = r4.apparent_encoding
tree4 = etree.HTML(r4.text)
soup4 = BeautifulSoup(r4.text, 'lxml')

In [16]:
paper_num = int(soup4.find('span', attrs={'id': 'footer_formatted_count'}).get_text().replace(',', ''))
print(paper_num)

109


In [17]:
k=''
for i in tree4.xpath("//div[@class='search-results-content']/div/div/a//text()"):
    k+=i
print(k)


Developing Renewable Agro-Based Anti-Icers for Sustainable Winter Road Maintenance Operations

Performance of FRP confined and unconfined engineered cementitious composite exposed to seawater

Novel mechanical models of tensile strength and elastic property of FDM AM PLA materials: Experimental and theoretical analyses

Silica Fume Effect on Engineering Properties of Superfine Cement-Grouted Sands

Effects of fiber orientation and content on the static and fatigue behavior of SFRC by using CT-Scan technology

Primary investigation on the relationship between microstructural characteristics and the mechanical performance of asphalt mixtures with different compaction degrees

Effect of mineral fillers on the performance, rheological and dynamic viscosity measurements of asphalt mastic

Behaviour of oil palm broom fibres (OPBF) reinforced concrete

Smart Semi-active PID-ACO control strategy for tower vibration reduction in Wind Turbines with MR damper

Mechanical properties of 16 differe

In [18]:
# NowCit_href=soup4.find('div', class_='create-cite-report').find('a')['href']
# wos_url='https://apps.webofknowledge.com'
# NowCit_url=wos_url+NowCit_href
# print(NowCit_url)
# r5 = requests.get(Citing_articles_href, 3)
# r5.encoding = r5.apparent_encoding
# tree5 = etree.HTML(r5.text)
# soup5 = BeautifulSoup(r5.text, 'lxml')

https://apps.webofknowledge.comOutboundService.do?action=go&mode=raService&queryOption(summary_search_mode)=TotalCitingArticles&SID=5CEtCHrcyrUirKesVea&product=WOS&parentProduct=WOS&parentQid=4&colName=WOS&MARKEDLIST=0


In [24]:
rurl='https://apps.webofknowledge.com/summary.do?mode=Analyze&product=WOS&product=WOS&search_mode=Analyze&parentQid='+str(qid+1)+'&qid='+str(qid+2)+'&SID='+sid+'&SID='+sid

headers3 = {
    'host':'apps.webofknowledge.com',
    'Origin': 'https://apps.webofknowledge.com',
    'Referer': 'https://apps.webofknowledge.com/InboundService.do?action=search&exclude=',
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    'Content-Type': 'application/x-www-form-urlencoded'
        }
fd3 = {
    'viewType':'summary',
    'product':'WOS',
    'rurl':quote(rurl, 'utf-8'),
    'mark_id':'WOS',
    'colName':'WOS',
    'search_mode':'Analyze',
    'locale':'en_US',
    'view_name':'WOS-Analyze-summary',
    'sortBy':'LC.A;PY.D;AU.A.en;SO.A.en;VL.D;PG.A',
    'mode':'OpenOutputService',
    'qid':qid+2,
    'SID':sid,
    'format':'saveToFile',
    'filters':'HIGHLY_CITED HOT_PAPER OPEN_ACCESS PMID USAGEIND AUTHORSIDENTIFIERS ACCESSION_NUM FUNDING SUBJECT_CATEGORY JCR_CATEGORY LANG IDS PAGEC SABBR CITREFC ISSN PUBINFO KEYWORDS CITTIMES ADDRS CONFERENCE_SPONSORS DOCTYPE ABSTRACT CONFERENCE_INFO SOURCE TITLE AUTHORS',
    'mark_to':109,
    'mark_from':1,
    'queryNatural':'<b>PUBLICATION NAME:</b> (frontiers of structural and civil engineering)',
    'count_new_items_marked':0,
    'use_two_ets':'false',
    'IncitesEntitled':'yes',
    'value(record_select_type)':'range',
    'markFrom':1,
    'markTo':109,
    'fields_selection':'HIGHLY_CITED HOT_PAPER OPEN_ACCESS PMID USAGEIND AUTHORSIDENTIFIERS ACCESSION_NUM FUNDING SUBJECT_CATEGORY JCR_CATEGORY LANG IDS PAGEC SABBR CITREFC ISSN PUBINFO KEYWORDS CITTIMES ADDRS CONFERENCE_SPONSORS DOCTYPE ABSTRACT CONFERENCE_INFO SOURCE TITLE AUTHORS',
    'save_options':'tabWinUTF8',
        }

In [26]:
s = requests.Session()
url='https://apps.webofknowledge.com/OutboundService.do'
r5 = s.post(url, data=fd3, headers=headers3)
with open('1.txt', 'wb') as f:
    f.write(r5.content)